In [1]:
import gzip
import ast
import json
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

vamos a traer el primer Archivo JSON. llamado output_steam_games

In [2]:


# Leer el archivo JSON línea por línea y cargar cada objeto JSON en la lista 'data'
data = []
with open('D:/USUARIO/Desktop/datos/output_steam_games.json', 'r') as file:
    for line in file:
        try:
            obj = json.loads(line)
            data.append(obj)
        except json.JSONDecodeError:
            print(f"Error al decodificar JSON en la línea: {line}")

vamos almacenar el dataframe con el nombre de steam_games_df

In [3]:
steam_games_df =pd.DataFrame(data)

In [4]:
filas_nulas = steam_games_df.isnull().all(axis=1).sum()
print(filas_nulas)

88310


Eliminamos las filas nulas de nuesto steam_games_df

In [5]:
steam_games_df= steam_games_df.dropna()


Vamos a trabajar con el archivo JSON llamado australian_user_reviews

In [6]:
with open('D:/USUARIO/Desktop/datos/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data = [eval(line) for line in file]
    
user_reviews = pd.DataFrame(data)

Luego de tener el archivo requerimos hacer el explode, esto con la finalidad de desanidar la columna reviews, luego la normalizamos para que tenga cada columna. para el final hacemos el concat de los dos dataset y eliminamos la comulna reviewa, y nos queda el nuevo dataSet

In [7]:
user_reviews_explored = user_reviews.explode('reviews')
user_reviews_normalized = pd.json_normalize(user_reviews_explored['reviews'])
user_reviews_explored.reset_index(drop=True, inplace=True)
user_reviews_normalized.reset_index(drop=True, inplace=True)
user_reviews_final = pd.concat([user_reviews_explored.drop('reviews',axis=1), user_reviews_normalized], axis=1)

Hacemos el mismo proceso para el archivo llamado australian_users_items. lo extraemos del archivo JSON

In [8]:
with open('D:/USUARIO/Desktop/datos/australian_users_items.json', 'r', encoding='utf-8') as file:
    data2 = [eval(line) for line in file]
    
user_items = pd.DataFrame(data2)

procedemos a desanidar el dataframe en la columna items

In [9]:
user_items_explored = user_items.explode('items')
user_items_normalized = pd.json_normalize(user_items_explored['items'])
user_items_explored.reset_index(drop=True, inplace=True)
user_items_normalized.reset_index(drop=True, inplace=True)
user_items_final = pd.concat([user_items_explored.drop('items',axis=1), user_items_normalized], axis=1)

In [11]:
user_items_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


Vamos a tomar este dataframe  para hacer un groupby, esto con el fin de tener las horas acumuladas  (playtime_forever)  por cada item_id.

In [12]:
df_agrupado = user_items_final.groupby(['item_id'])['playtime_forever'].sum().reset_index().rename(columns={'playtime_forever':'suma_playtime_forever'})


In [13]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10978 entries, 0 to 10977
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   item_id                10978 non-null  object 
 1   suma_playtime_forever  10978 non-null  float64
dtypes: float64(1), object(1)
memory usage: 171.7+ KB


Vamos a trabajar con el dataframe  user_review_final. primero vamos a crear la columna 'sentiment-analysis' esto lo vamos hacer con la libreria Nnltk que sirve para esudio de analisis de sentimiento, de acuerdo a los   valores que se veran reflejados en la columna, haremos luegola conversión cmo lo piden en el enunciado.

In [14]:
sid = SentimentIntensityAnalyzer()

In [15]:
user_reviews_final['sentiment-analysis'] = user_reviews_final['review'].apply(lambda x: sid.polarity_scores(str(x))['compound'] if isinstance(x, str) else 0)

aca vamos hacer el cambio de acuerdo al enunciado: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews

In [16]:
user_reviews_final.loc[user_reviews_final['sentiment-analysis'] > 0, 'sentiment-analysis'] = 2
user_reviews_final.loc[user_reviews_final['sentiment-analysis'] == 0, 'sentiment-analysis'] = 1
user_reviews_final.loc[user_reviews_final['sentiment-analysis'] < 0, 'sentiment-analysis'] = 0

In [17]:
user_reviews_final.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment-analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2.0
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2.0
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2.0


Procedemos a eliminar la columna review.

In [18]:
user_reviews_final = user_reviews_final.drop('review',axis=1)

In [19]:
user_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             59333 non-null  object 
 1   user_url            59333 non-null  object 
 2   funny               59305 non-null  object 
 3   posted              59305 non-null  object 
 4   last_edited         59305 non-null  object 
 5   item_id             59305 non-null  object 
 6   helpful             59305 non-null  object 
 7   recommend           59305 non-null  object 
 8   sentiment-analysis  59333 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.1+ MB


In [20]:
conteo_valores = user_reviews_final['sentiment-analysis'].value_counts()
print(conteo_valores)

sentiment-analysis
2.0    37657
1.0    11844
0.0     9832
Name: count, dtype: int64


Procedemos a unir el user_reviews_final con df_agrupado haciendo un merge con la columna en comun 'item_id'

In [21]:
df_merge1 = pd.merge(df_agrupado,user_reviews_final,on='item_id')

ahora para concluir debemos hacer la union con el dataframe faltante llamado steam_games_df, pero antes debemos cmabiar el nombre de  'id'po 'item_id'

In [22]:
steam_games_df = steam_games_df.rename(columns={'id': 'item_id'})

ahora si vamos hacer el merge. de steam_games_df con df:merge1

In [23]:
consolidado_df = pd.merge(df_merge1 ,steam_games_df,on='item_id')

In [27]:
consolidado_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44680 entries, 0 to 44679
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   item_id                44680 non-null  object 
 1   suma_playtime_forever  44680 non-null  float64
 2   user_id                44680 non-null  object 
 3   user_url               44680 non-null  object 
 4   funny                  44680 non-null  object 
 5   posted                 44680 non-null  object 
 6   last_edited            44680 non-null  object 
 7   helpful                44680 non-null  object 
 8   recommend              44680 non-null  object 
 9   sentiment-analysis     44680 non-null  float64
 10  publisher              44680 non-null  object 
 11  genres                 44680 non-null  object 
 12  app_name               44680 non-null  object 
 13  title                  44680 non-null  object 
 14  url                    44680 non-null  object 
 15  re

In [28]:
consolidado_df.head()

,item_id,suma_playtime_forever,user_id,user_url,funny,posted,last_edited,helpful,recommend,sentiment-analysis,...,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,developer
0,10,17386015.0,Bennysaputra,http://steamcommunity.com/id/Bennysaputra,,"Posted August 1, 2015.",,No ratings yet,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
1,10,17386015.0,Monsta45,http://steamcommunity.com/id/Monsta45,,"Posted April 26, 2015.",,No ratings yet,True,1.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
2,10,17386015.0,76561198040188061,http://steamcommunity.com/profiles/76561198040...,,"Posted May 18, 2011.",,0 of 2 people (0%) found this review helpful,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
3,10,17386015.0,mayshowganmore,http://steamcommunity.com/id/mayshowganmore,,"Posted January 22, 2014.",,0 of 2 people (0%) found this review helpful,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
4,10,17386015.0,BestinTheWorldThund3r,http://steamcommunity.com/id/BestinTheWorldThu...,,"Posted March 19, 2014.",,0 of 2 people (0%) found this review helpful,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve


In [26]:
consolidado_df.to_csv('padre_df1.csv',index = False)